In [10]:
from astroquery.mast import Observations
import numpy as np

In [22]:
proposal_id = 15354

# Query MAST for observations
obs_table = Observations.query_criteria(
    obs_collection="HST",
    proposal_id=[f"{proposal_id}"],
    dataRights="PUBLIC",
)

print(len(obs_table))

# Get preview products
products = Observations.get_product_list(obs_table)
products = products[products["productType"] == "PREVIEW"]

match = "total"
exclude = "color"
match_mask = [match in row["productFilename"] for row in products]
exclude_mask = [exclude not in row["productFilename"] for row in products]
mask = [m and e for m, e in zip(match_mask, exclude_mask)]

seed = 42
n_max_images = 20

products = products[mask][np.random.RandomState(seed=seed).choice(np.arange(len(products[mask])), n_max_images)]

print(len(products))

5
20


In [23]:
import os

download_dir = 'download_dir'

Observations.download_products(
    products,
    extension=["jpg", "jpeg"],
    productType="PREVIEW",
    download_dir=download_dir,
)

# Recursively find all images in "total_images" directory and bring them up to "total_images"
for root, dirs, files in os.walk(download_dir):
    for file in files:
        if file.endswith((".jpg", ".jpeg")):
            source = os.path.join(root, file)
            destination = os.path.join(download_dir, file)

            # Check if file already exists in the destination
            if not os.path.exists(destination):
                os.rename(source, destination)


INFO: 19 of 20 products were duplicates. Only downloading 1 unique product(s). [astroquery.mast.observations]
